# Multimodal AI app for videos using Twelve Labs

In three easy steps:

1) pip install twelvelabs
2) Create twelvelabs client to generate video embeddings
3) Use Snowflake's VECTOR_COSINE_SIMILARITY function to find (similar) videos based on text query

Prerequisites: 
> *Create [network rule](https://docs.snowflake.com/en/sql-reference/sql/create-network-rule) and [external access integration](https://docs.snowflake.com/en/sql-reference/sql/create-external-access-integration) (EAI) for installing `twelvelabs` Python packages* and [enable](https://docs.snowflake.com/en/user-guide/ui-snowsight/notebooks-external-access#enable-existing-external-access-integrations-eai) it for this notebook.

> *Obtain a [Twelve Labs API key](https://api.twelvelabs.io).*


## What is Container Runtime? 

It provides a flexible environment to build and operationalize a variety of workloads, especially hashtag#ML that require Python packages from multiple sources and powerful compute — both CPUs and GPUs. With this Snowflake-native experience, you can also train models, perform hyperparameter tuning as well as batch inference.

Here are some resources for you to get started:

👉 [Documentation](https://docs.snowflake.com/en/user-guide/ui-snowsight/notebooks-on-spcs) includes quickstart guides

👉 [Github repo](https://github.com/Snowflake-Labs/snowflake-demo-notebooks) of tutorials and example notebooks

👉 [YouTube](https://www.youtube.com/playlist?list=PLavJpcg8cl1Efw8x_fBKmfA2AMwjUaeBI) demo playlist

In [ ]:
!pip install twelvelabs

In [ ]:
from twelvelabs import TwelveLabs
from twelvelabs.models.embed import EmbeddingsTask
from snowflake.snowpark.context import get_active_session
session = get_active_session()
TWELVE_LABS_API_KEY ="tlk_2VAE3AA2XGRHKXXXXXXXXXXXX"
# Initialize the Twelve Labs client
twelvelabs_client = TwelveLabs(api_key=TWELVE_LABS_API_KEY)
def get_video_embeddings(url: str) -> float:
    twelvelabs_client = TwelveLabs(api_key=TWELVE_LABS_API_KEY)
    task = twelvelabs_client.embed.task.create(
        engine_name="Marengo-retrieval-2.6",
        video_url=url)
    task.wait_for_done()
    task_result = twelvelabs_client.embed.task.retrieve(task.id)
    embeddings = []
    for v in task_result.video_embeddings:
        embeddings.append({
            'embedding': v.embedding.float,
            'start_offset_sec': v.start_offset_sec,
            'end_offset_sec': v.end_offset_sec,
            'embedding_scope': v.embedding_scope
        })
    return embeddings if embeddings else None

video_url = 'https://videos.s3.us-west-1.amazonaws.com/misc/videos/demo.mp4'
embeddings = get_video_embeddings(video_url)

In [ ]:
def get_similar_videos(text_query):
    # Twelve Labs Embed API supports text-to-embedding  
    twelvelabs_response = twelvelabs_client.embed.create(
      engine_name="Marengo-retrieval-2.6",
      text=text_query,
      text_truncate="start"
    )

    text_query_embeddings = None
    if twelvelabs_response.text_embedding is not None and twelvelabs_response.text_embedding.segments is not None:
        text_query_embeddings = twelvelabs_response.text_embedding.segments[0].embeddings_float

    if text_query_embeddings:
      return session.sql(f"""SELECT 
          round(VECTOR_COSINE_SIMILARITY({embeddings[0]['embedding']}::VECTOR(FLOAT, 1024),
          {text_query_embeddings}::VECTOR(FLOAT, 1024)),2) as similarity""") \
      .to_pandas().iloc[0]['SIMILARITY']
    else:
      return "N/A"

get_similar_videos('snowflake notebooks')